# Week 2 Assignment: CIFAR-10 Autoencoder

For this week, you will create a convolutional autoencoder for the [CIFAR10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset. You are free to choose the architecture of your autoencoder provided that the output image has the same dimensions as the input image.

After training, your model should meet loss and accuracy requirements when evaluated with the test dataset. You will then download the model and upload it in the classroom for grading.

Let's begin!

***Important:*** *This colab notebook has read-only access so you won't be able to save your changes. If you want to save your work periodically, please click `File -> Save a Copy in Drive` to create a copy in your account, then work from there.*  

## Imports

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import Sequential

# 2.15.0
print(tf.__version__)
# 4.9.4
print(tfds.__version__)

d:\Studi\MBKM\bangkit-machine-learning\venv\tf2_16_1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.16.1
4.9.4


## Load and prepare the dataset

The [CIFAR 10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset already has train and test splits and you can use those in this exercise. Here are the general steps:

* Load the train/test split from TFDS. Set `as_supervised` to `True` so it will be convenient to use the preprocessing function we provided.
* Normalize the pixel values to the range [0,1], then return `image, image` pairs for training instead of `image, label`. This is because you will check if the output image is successfully regenerated after going through your autoencoder.
* Shuffle and batch the train set. Batch the test set (no need to shuffle).


In [3]:
# preprocessing function
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0

  return image, image # dataset label is not used. replaced with the same image input.

# parameters
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024


### START CODE HERE (Replace instances of `None` with your code) ###

# use tfds.load() to fetch the 'train' split of CIFAR-10
(train_dataset, _), (test_dataset, _) = tf.keras.datasets.cifar10.load_data()
train_dataset = tf.data.Dataset.from_tensor_slices((train_dataset, train_dataset))
test_dataset = tf.data.Dataset.from_tensor_slices((test_dataset, test_dataset))
# tfds.load('cifar10', split='train', as_supervised=True)

# preprocess the dataset with the `map_image()` function above
train_dataset = train_dataset.map(map_image)

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)


# use tfds.load() to fetch the 'test' split of CIFAR-10
# test_dataset = tfds.load('cifar10', split='test', as_supervised=True)

# preprocess the dataset with the `map_image()` function above
test_dataset = test_dataset.map(map_image)

# batch the dataset
test_dataset = test_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

print(len(train_dataset) // BATCH_SIZE)
print(len(test_dataset) // BATCH_SIZE)

### END CODE HERE ###

3
0


## Build the Model

Create the autoencoder model. As shown in the lectures, you will want to downsample the image in the encoder layers then upsample it in the decoder path. Note that the output layer should be the same dimensions as the original image. Your input images will have the shape `(32, 32, 3)`. If you deviate from this, your model may not be recognized by the grader and may fail.

We included a few hints to use the Sequential API below but feel free to remove it and use the Functional API just like in the ungraded labs if you're more comfortable with it. Another reason to use the latter is if you want to visualize the encoder output. As shown in the ungraded labs, it will be easier to indicate multiple outputs with the Functional API. That is not required for this assignment though so you can just stack layers sequentially if you want a simpler solution.

In [4]:
# suggested layers to use. feel free to add or remove as you see fit.
from keras.layers import Conv2D, UpSampling2D, MaxPooling2D, Input

# use the Sequential API (you can remove if you want to use the Functional API)
model = Sequential()

### START CODE HERE ###
# use `model.add()` to add layers (if using the Sequential API)
def encoder(inputs):
    conv2d_1 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    max_pool_1 = MaxPooling2D((2, 2))(conv2d_1)
    
    conv2d_2 = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(max_pool_1)
    max_pool_2 = MaxPooling2D((2, 2))(conv2d_2)
    
    return max_pool_2

def bottleneck(inputs):
    bottleneck = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    encoder_visualization = Conv2D(filters=1, kernel_size=(3, 3), activation='sigmoid', padding='same')(bottleneck)
    
    return bottleneck, encoder_visualization

def decoder(inputs):
    conv2d_1 = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    upsample_1 = UpSampling2D((2, 2))(conv2d_1)
    
    conv2d_2 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(upsample_1)
    upsample_2 = UpSampling2D((2, 2))(conv2d_2)
    
    conv2d_3 = Conv2D(filters=3, kernel_size=(3, 3), activation='sigmoid', padding='same')(upsample_2)
    
    return conv2d_3
    
inputs = Input(shape=(32, 32, 3))
encoder_output = encoder(inputs=inputs)
bottleneck_output, encoder_visualization = bottleneck(inputs=encoder_output)
decoder_output = decoder(bottleneck_output)

model = tf.keras.Model(inputs=inputs, outputs=decoder_output)
encoder_model = tf.keras.Model(inputs=inputs, outputs=encoder_visualization)

### END CODE HERE ###

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 128)      │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 16, 16, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 3)      │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 741,379 (2.83 MB)

 Trainable params: 741,379 (2.83 MB)

 Non-trainable params: 0 (0.00 B)

## Configure training parameters

We have already provided the optimizer, metrics, and loss in the code below.

In [5]:
# Please do not change the model.compile() parameters
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

## Training

You can now use [model.fit()](https://keras.io/api/models/model_training_apis/#fit-method) to train your model. You will pass in the `train_dataset` and you are free to configure the other parameters. As with any training, you should see the loss generally going down and the accuracy going up with each epoch. If not, please revisit the previous sections to find possible bugs.

*Note: If you get a `dataset length is infinite` error. Please check how you defined `train_dataset`. You might have included a [method that repeats the dataset indefinitely](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#repeat).*

In [8]:
# parameters (feel free to change this)
# hanya terbatas pada accuracy==0.7 dan loss==0.005
train_steps = len(train_dataset) // BATCH_SIZE
val_steps = len(test_dataset) // BATCH_SIZE

# dapat mencapai accuracy 0.85 dan loss==0.0013
# train_steps = tf.data.experimental.cardinality(train_dataset).numpy()
# val_steps = tf.data.experimental.cardinality(test_dataset).numpy()

### START CODE HERE ###
# model.fit(train_dataset, steps_per_epoch=train_steps, validation_data=test_dataset, validation_steps=val_steps, epochs=60)
### END CODE HERE ###

391
79


## Model evaluation

You can use this code to test your model locally before uploading to the grader. To pass, your model needs to satisfy these two requirements:

* loss must be less than 0.01
* accuracy must be greater than 0.6

In [12]:
result = model.evaluate(test_dataset, steps=10)

 5/10 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.8109 - loss: 0.0050

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.8108 - loss: 0.0050


If you did some visualization like in the ungraded labs, then you might see something like the gallery below. This part is not required.

<img src="https://storage.googleapis.com/tensorflow-3-public/assets/images/autoencoder.png" width="75%" height="75%"/>

## Save the Model

Once you're satisfied with the results, you can save your model and upload it to the grader in Coursera. Please run all succeeding cells to ensure that you will have a gradable submission. Otherwise, you might get this error message:

`There was a problem grading your submission. Check stderr for more details.`

First, save the model file in your Colab workspace.

In [13]:
# Save the model you just trained
model.save("temp_model.h5")

Tensorflow releases minor and patch updates every few months ([sometimes quicker](https://pypi.org/project/tensorflow/#history)). Oftentimes, the code and syntax won't change but the newer versions will have a different model definition under the hood. The next cells will convert your newer model to one that the grader can read. You won't have to do this in your personal projects. This is just an extra step for grading.

In [ ]:
# Install packages for compatibility with the autograder

# NOTE: You can safely ignore errors about version incompatibility of
# Colab-bundled packages (e.g. xarray, pydantic, etc.)

!pip install tensorflow==2.8.0 --quiet
!pip install keras==2.8.0 --quiet

Wait for the cell above to complete. After that, **go to `Runtime > Restart Session` and wait for the kernel to reconnect.** This is necessary so you can reload Tensorflow with the version installed above. Run the next cell to check if you have the correct version. You will get an `AssertionError` if you didn't restart the session.

In [1]:
import tensorflow as tf

# Check if you have the correct Tensorflow version
assert tf.__version__ == '2.8.0', f'You have TF{tf.__version__}. Please install the grader-compatible Tensorflow and select Runtime > Restart Session'

If you didn't get an error above, you can finally convert the model.

In [7]:
print(tf.__version__)

# Load the model you saved earlier
# model = tf.keras.models.load_model("./temp_model.h5", compile=False)
model = tf.keras.models.load_model("temp_model.h5")

# For this assignment only. The model has to be compiled with these settings.
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

# Save the model with the compatible TF version
model.save("mymodel.h5")

2.8.0


ValueError: Unrecognized keyword arguments: ['batch_shape']

Please look for `mymodel.h5` in the File Explorer on the left and download it. Then go back to the Coursera classroom and upload it to `My Submissions` tab of the Week 2 assignment. Alternatively, you can also use the cell below to download the model.



In [ ]:
# You can also use this cell as a shortcut for downloading your model
from google.colab import files
files.download("mymodel.h5")

**Congratulations on completing this week's assignment!**